# TMAE Analysis

Source: https://dadosabertos.aneel.gov.br/dataset/atendimento-ocorrencias-emergenciais

Last Update: 2026/02/17


## Libs:

In [1]:
! pip install --quiet requests 
! pip install --quiet pandas
! pip install --quiet numpy
! pip install --quiet seaborn
! pip install --quiet matplotlib
! pip install --quiet scikit-learn
! pip install --quiet scipy
import requests
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Patch
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import datetime as dt

## Preparing Enviroment:

In [2]:
# Config - Df config:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None) 
pd.set_option("display.float_format", "{:.4f}".format)

In [3]:
# Config - Image folder:
PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
os.makedirs(IMAGES_PATH, exist_ok=True)

In [4]:
# Function - Get data csv:
def get_data_csv(url_csv):
    df = pd.read_csv(url_csv, sep=";", encoding="latin1", decimal=',')
    return df

In [5]:
# Function - Get data API:
def get_data_api(resource_id):
    offset = 0
    total = []

    while True:
        url = f"https://dadosabertos.aneel.gov.br/api/3/action/datastore_search?resource_id={resource_id}&limit=10000&offset={offset}"
        result = requests.get(url)
        result.raise_for_status()    
        data_json = result.json()
        
        n = data_json['result']['records']

        if not n: 
            break  
        
        total.extend(n)
        offset += 10000
        print(f"Lines collected: {len(total)}")

        # Formação do df:
        df = pd.DataFrame(n)
    return df

In [6]:
# Function - Convert Columns:
def convert_dt_columns(df):
    # Dt Columns:
    columns_dt = [col for col in df.columns if col.upper().startswith('DT')]
    for col in columns_dt:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Dat Columns:
    columns_dat = [col for col in df.columns if col.upper().startswith('DAT')]
    for col in columns_dat:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return df

## Getting Database:

In [7]:
# Principal database:
df = get_data_csv('https://dadosabertos.aneel.gov.br/dataset/dcdff65d-d173-4d2e-a3b9-84eec7702f06/resource/73b00e68-66b1-4a72-8d72-b7baab47048c/download/indicador-atendimento-emergencial.csv')
df.columns = df.columns.str.upper()
df.head()

,DATGERACAOCONJUNTODADOS,SIGAGENTE,NUMCNPJ,IDECONJUNDCONSUMIDORAS,DSCCONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
0,05-02-2026,CPFL-PAULISTA,33050196000188.0000,25.0000,PIRACICABA,NDIACRI,2010,1,10.0000
1,05-02-2026,COELBA,15139629000194.0000,48.0000,MACAUBAS,NDIACRI,2010,1,15.0000
2,05-02-2026,ENEL CE,7047251000170.0000,49.0000,JUATAMA,NDIACRI,2010,1,0.0000
3,05-02-2026,NaN,19527639006601.0000,82.0000,TABULEIRO,NDIACRI,2010,1,3.0000
4,05-02-2026,EMR,19527639000158.0000,82.0000,TABULEIRO,NDIACRI,2010,1,3.0000


In [8]:
# Helpers - Databases - KPIs:
kpis = "e3724332-7de8-4273-8f0d-b20ebb91bf7c"
dfKPIs = get_data_api(kpis)
dfKPIs.columns = dfKPIs.columns.str.upper()
dfKPIs.head()

Lines collected: 468


,_ID,DATGERACAOCONJUNTODADOS,SIGINDICADOR,DSCINDICADOR
0,1,2022-08-17T00:00:00,AREA,"Área do conj., expressa em km2, correspondente a área geogr. e não a área elétr."
1,2,2022-08-17T00:00:00,AREAT,Área do conjunto em km2
2,3,2022-08-17T00:00:00,CM,Encargo de uso do sistema de distribuição aplicado à unidade cons. (mensal)
3,4,2022-08-17T00:00:00,CMA,Encargo de uso do sistema de distribuição aplicado à unidade consumidora (anual)
4,5,2022-08-17T00:00:00,CMM,"Consumo médio mensal, em MWh, média aritmética simples do último ano, excluin-"


In [9]:
# Helpers - Databases - Companies:
companies = "64250fc9-4f7a-4d97-b0d4-3c090e005e1c"
dfComp = get_data_api(companies)
dfComp.columns = dfComp.columns.str.upper()
dfComp.head()

Lines collected: 9736


,_ID,DATGERACAOCONJUNTODADOS,NUMCNPJ,SIGPESSOA,NOMRAZAOSOCIAL,IDCATIVO,IDCCOMERCIALIZACAO,IDCDISTRIBUICAO,IDCGERACAO,IDCTRANSMISSAO
0,1,2026-02-01,04890965000158,None,'' BG NORTE PETROLEO LTDA '',A,0,0,1,0
1,2,2026-02-01,32190231000100,None,2BR COMERCIALIZADORA DE ENERGIA LTDA,A,1,0,0,0
2,3,2026-02-01,03407182000108,None,2MS - ENGENHARIA LTDA,A,0,0,1,0
3,4,2026-02-01,36583766000193,None,2W COMERCIALIZADORA VAREJISTA DE ENERGIA S.A.,A,1,0,0,0
4,5,2026-02-01,08773135000100,None,2W ECOBANK S.A. - EM RECUPERACAO JUDICIAL,A,1,0,0,0


## Exploring Database:

In [10]:
# Database info:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672090 entries, 0 to 6672089
Data columns (total 9 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   DATGERACAOCONJUNTODADOS  object 
 1   SIGAGENTE                object 
 2   NUMCNPJ                  float64
 3   IDECONJUNDCONSUMIDORAS   float64
 4   DSCCONJUNDCONSUMIDORAS   object 
 5   SIGINDICADOR             object 
 6   ANOINDICE                int64  
 7   NUMPERIODOINDICE         int64  
 8   VLRINDICEENVIADO         float64
dtypes: float64(3), int64(2), object(4)
memory usage: 458.1+ MB


In [11]:
# Removing unnecessary columns:
cols_to_drop = ['DATGERACAOCONJUNTODADOS','NUMCNPJ']
df.drop(columns=cols_to_drop, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672090 entries, 0 to 6672089
Data columns (total 7 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SIGAGENTE               object 
 1   IDECONJUNDCONSUMIDORAS  float64
 2   DSCCONJUNDCONSUMIDORAS  object 
 3   SIGINDICADOR            object 
 4   ANOINDICE               int64  
 5   NUMPERIODOINDICE        int64  
 6   VLRINDICEENVIADO        float64
dtypes: float64(2), int64(2), object(3)
memory usage: 356.3+ MB


In [12]:
# Database describe:
df.describe()

,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
count,6672053.0000,6672090.0000,6672090.0000,6672090.0000
mean,10643.3344,2010.3956,6.4132,99.7818
std,4171.4301,7.6717,3.4899,297.6341
min,1.0000,2000.0000,1.0000,-7112.4000
25%,7815.0000,2004.0000,3.0000,5.3800
50%,11113.0000,2009.0000,6.0000,40.8100
75%,14158.0000,2017.0000,9.0000,106.8200
max,17419.0000,2026.0000,12.0000,371168.0000


For this analysis, we will focus on the most recent five years of data, which requires filtering out earlier records from the dataset. We will also exclude data from 2026, as it is incomplete (only January is available).

In [13]:
# Removing unnecessary years:
df = df[df['ANOINDICE'] >= 2020]
df = df[df['ANOINDICE'] <= 2025]
df.describe()

,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
count,1184828.0000,1184828.0000,1184828.0000,1184828.0000
mean,14916.1878,2022.5088,6.3339,174.9401
std,1422.5731,1.7104,3.5269,227.0472
min,964.0000,2020.0000,1.0000,0.0000
25%,13753.0000,2021.0000,3.0000,40.8900
50%,14976.0000,2023.0000,6.0000,95.0000
75%,16142.0000,2024.0000,9.0000,234.0000
max,17419.0000,2025.0000,12.0000,78220.4700


Based on the number of records in each column, we can conclude that there are no missing values. Now, let us analyze the meaning of each KPI in the SIGINDICADOR column and select the ones that will be used in this study.

In [17]:
# Merging dataframes: database + kpis:
dfMod = pd.merge(df, dfKPIs, on=['SIGINDICADOR'], how='left')
dfMod.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,DSCCONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,_ID,DATGERACAOCONJUNTODADOS,DSCINDICADOR
0,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2025,1,5.0000,245,2022-08-17T00:00:00,Número de dias críticos verificados no ano
1,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2024,1,5.0000,245,2022-08-17T00:00:00,Número de dias críticos verificados no ano
2,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2021,1,11.0000,245,2022-08-17T00:00:00,Número de dias críticos verificados no ano
3,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2022,1,12.0000,245,2022-08-17T00:00:00,Número de dias críticos verificados no ano
4,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2023,1,12.0000,245,2022-08-17T00:00:00,Número de dias críticos verificados no ano


In [18]:
# Removing unnecessary columns:
cols_to_drop = ['DATGERACAOCONJUNTODADOS','_ID']
dfMod.drop(columns=cols_to_drop, inplace=True)
dfMod.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,DSCCONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR
0,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2025,1,5.0000,Número de dias críticos verificados no ano
1,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2024,1,5.0000,Número de dias críticos verificados no ano
2,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2021,1,11.0000,Número de dias críticos verificados no ano
3,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2022,1,12.0000,Número de dias críticos verificados no ano
4,CPFL JAGUARI,964.0000,PEDREIRA,NDIACRI,2023,1,12.0000,Número de dias críticos verificados no ano


In [22]:
# Analyzing which indicators are present in the SIGINDICADOR column:
dfMod.groupby(['SIGINDICADOR','DSCINDICADOR']).size().reset_index(name='REGISTERS')

,SIGINDICADOR,DSCINDICADOR,REGISTERS
0,NDIACRI,Número de dias críticos verificados no ano,17947
1,Nie,Número de Ocorreências Emergenciais com Interrupções,229783
2,NumOcorr,Número de Ocorrencias Emergenciais no Conjunto no Período,229785
3,TMD,Tempo Médio de Deslocamento,229794
4,TME,Tempo Médio de Execução,229794
5,TMP,Tempo Médio de Preparação,229793
6,VLCLACRI,Valor utilizado no ano para identificar se um dia pode ser classificado critico,17932


Indicators related to Critical Day (NDIACRI and VLCLACRI) will not be part of this study, since our goal is to analyze only emergency service responses. Therefore, rows containing these indicators will be removed.

In [23]:
# Removing unnecessary KPIs:
dfMod.drop(dfMod[dfMod['SIGINDICADOR'].isin(['NDIACRI'])].index, inplace=True)
dfMod.drop(dfMod[dfMod['SIGINDICADOR'].isin(['VLCLACRI'])].index, inplace=True)
dfMod.head()

,SIGAGENTE,IDECONJUNDCONSUMIDORAS,DSCCONJUNDCONSUMIDORAS,SIGINDICADOR,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO,DSCINDICADOR
35879,CPFL JAGUARI,964.0000,PEDREIRA,TME,2024,10,40.0700,Tempo Médio de Execução
35880,CPFL JAGUARI,964.0000,PEDREIRA,TME,2021,2,66.8600,Tempo Médio de Execução
35881,CPFL JAGUARI,964.0000,PEDREIRA,TME,2020,5,55.3800,Tempo Médio de Execução
35882,CPFL JAGUARI,964.0000,PEDREIRA,TME,2023,12,47.4700,Tempo Médio de Execução
35883,CPFL JAGUARI,964.0000,PEDREIRA,TME,2021,3,36.1900,Tempo Médio de Execução


In [24]:
print('Database info after cleaning:')
dfMod.info()
print('\n Database describe after cleaning:')
dfMod.describe()

Database info after cleaning:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1148949 entries, 35879 to 1184827
Data columns (total 8 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SIGAGENTE               1133649 non-null  object 
 1   IDECONJUNDCONSUMIDORAS  1148949 non-null  float64
 2   DSCCONJUNDCONSUMIDORAS  1148949 non-null  object 
 3   SIGINDICADOR            1148949 non-null  object 
 4   ANOINDICE               1148949 non-null  int64  
 5   NUMPERIODOINDICE        1148949 non-null  int64  
 6   VLRINDICEENVIADO        1148949 non-null  float64
 7   DSCINDICADOR            1148949 non-null  object 
dtypes: float64(2), int64(2), object(4)
memory usage: 70.1+ MB

 Database describe after cleaning:


,IDECONJUNDCONSUMIDORAS,ANOINDICE,NUMPERIODOINDICE,VLRINDICEENVIADO
count,1148949.0000,1148949.0000,1148949.0000,1148949.0000
mean,14916.4839,2022.5074,6.5005,179.9106
std,1422.7862,1.7111,3.4512,228.7718
min,964.0000,2020.0000,1.0000,0.0000
25%,13753.0000,2021.0000,4.0000,43.3800
50%,14976.0000,2023.0000,6.0000,101.0000
75%,16144.0000,2024.0000,9.0000,240.0000
max,17419.0000,2025.0000,12.0000,78220.4700


Now we need to pivot the KPI rows into separate columns, one column per KPI, using VLRINDICEENVIADO as the values.